In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

data\sample.txt
data\test.txt
data\train.txt


In [9]:
import re
import pandas as pd

path = "data/train.txt"  # đổi nếu cần

rows = []
with open(path, encoding="utf-8") as f:
    for line in f:
        nums = re.findall(r"-?\d+", line)
        if len(nums) >= 3:
            rows.append([int(nums[0]), int(nums[1]), int(nums[2])])

df_train = pd.DataFrame(rows, columns=["userid", "movieid", "rating"])
print(df_train.head())
print(df_train.dtypes)

   userid  movieid  rating
0       1        1       5
1       1        2       3
2       1        3       4
3       1        4       3
4       1        5       3
userid     int64
movieid    int64
rating     int64
dtype: object


In [10]:
path = "data/test.txt"  # đổi nếu cần

rows = []
with open(path, encoding="utf-8") as f:
    for line in f:
        nums = re.findall(r"-?\d+", line)
        if len(nums) >= 2:
            rows.append([int(nums[0]), int(nums[1])])

df_test = pd.DataFrame(rows, columns=["userid", "movieid"])
print(df_test.head())
print(df_test.dtypes)

   userid  movieid
0       1       20
1       1       33
2       1       61
3       1      117
4       1      155
userid     int64
movieid    int64
dtype: object


In [11]:
import torch
import torch.nn.functional as F
from torch import nn
from typing import Sequence, List
class NeuMF(nn.Module):
    """
    NeuMF với one-hot embedding:
      - User/Item -> one-hot (cố định) -> Linear chiếu sang latent (thay cho nn.Embedding)
      - Nhánh GMF: element-wise product giữa 2 latent vector
      - Nhánh MLP: concat 2 latent, qua nhiều Dense + ReLU (+ Dropout)
      - Fusion: concat(GMF, MLP) -> Dropout -> (Dropout head) -> Linear(…, 1)

    Tham số dropout:
      - dropout_proj  : dropout ngay sau chiếu one-hot -> latent (cả GMF và MLP)
      - dropout_hidden: dropout sau mỗi Dense của tháp MLP
      - dropout_fusion: dropout sau khi concat GMF & MLP
      - dropout_fc    : dropout ngay trước lớp Linear cuối cùng
    """
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 k_gmf: int = 8,
                 k_mlp: int = 32,
                 mlp_layers=(64, 32, 16),
                 dropout_hidden: float = 0.3,
                 dropout_proj: float = 0.1,
                 dropout_fusion: float = 0.2,
                 dropout_fc: float = 0.2):
        super().__init__()

        # One-hot "embeddings": identity, freeze để dùng như one-hot lookup
        self.oh_user = nn.Embedding.from_pretrained(torch.eye(num_users), freeze=True)
        self.oh_item = nn.Embedding.from_pretrained(torch.eye(num_items), freeze=True)

        # Chiếu one-hot -> latent (thay cho nn.Embedding)
        self.gmf_user_proj = nn.Linear(num_users, k_gmf, bias=False)
        self.gmf_item_proj = nn.Linear(num_items, k_gmf, bias=False)
        self.mlp_user_proj = nn.Linear(num_users, k_mlp, bias=False)
        self.mlp_item_proj = nn.Linear(num_items, k_mlp, bias=False)

        # Dropout sau projection
        self.do_proj_gmf_u = nn.Dropout(dropout_proj)
        self.do_proj_gmf_i = nn.Dropout(dropout_proj)
        self.do_proj_mlp_u = nn.Dropout(dropout_proj)
        self.do_proj_mlp_i = nn.Dropout(dropout_proj)

        # Tháp MLP: (Linear -> ReLU -> Dropout) * len(mlp_layers)
        mlp = []
        in_dim = k_mlp * 2
        for units in mlp_layers:
            mlp += [
                nn.Linear(in_dim, units),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout_hidden),
            ]
            in_dim = units
        self.mlp_layers = nn.Sequential(*mlp)

        # Dropout sau khi concat(GMF, MLP)
        self.do_fusion = nn.Dropout(dropout_fusion)

        # Head: Dropout trước fc
        fusion_dim = k_gmf + (mlp_layers[-1] if mlp_layers else k_mlp * 2)
        self.head = nn.Sequential(
            nn.Dropout(dropout_fc),
            nn.Linear(fusion_dim, 1)
        )

        self._init_weights()

    def _init_weights(self):
        # Xavier cho các Linear
        for m in [self.gmf_user_proj, self.gmf_item_proj, self.mlp_user_proj, self.mlp_item_proj]:
            nn.init.xavier_uniform_(m.weight)
        for m in self.mlp_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)
        lin = self.head[1]  # Linear cuối
        nn.init.xavier_uniform_(lin.weight)
        nn.init.zeros_(lin.bias)

    def forward(self, users: torch.Tensor, items: torch.Tensor) -> torch.Tensor:
        """
        users: LongTensor shape (B,)
        items: LongTensor shape (B,)
        return: FloatTensor shape (B,)  — điểm số (rating dự đoán)
        """
        # One-hot vectors (0/1)
        u_oh = self.oh_user(users)   # (B, num_users)
        i_oh = self.oh_item(items)   # (B, num_items)

        # ----- GMF path -----
        gu = self.gmf_user_proj(u_oh)      # (B, k_gmf)
        gi = self.gmf_item_proj(i_oh)      # (B, k_gmf)
        gu = self.do_proj_gmf_u(gu)
        gi = self.do_proj_gmf_i(gi)
        gmf = gu * gi                      # element-wise

        # ----- MLP path -----
        mu = self.mlp_user_proj(u_oh)      # (B, k_mlp)
        mi = self.mlp_item_proj(i_oh)      # (B, k_mlp)
        mu = self.do_proj_mlp_u(mu)
        mi = self.do_proj_mlp_i(mi)
        x = torch.cat([mu, mi], dim=-1)    # (B, 2*k_mlp)
        x = self.mlp_layers(x)

        # ----- Fusion + head -----
        z = torch.cat([gmf, x], dim=-1)
        z = self.do_fusion(z)
        score = self.head(z).squeeze(-1)   # (B,)
        return score

In [12]:
import torch
import torch.nn.functional as F
from torch import nn
from typing import Sequence, List

class DMF(nn.Module):
    """
    Deep Matrix Factorization (học trực tiếp A):
      - A: (num_users, num_items) là tham số học được.
      - forward(users, items):
          + user vector = A[u, :]  (1 hàng)
          + item vector = A[:, v]  (1 cột)
          + qua MLP riêng -> L2-normalize -> cosine -> map [1,5]
    """
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 d1: int = 64,                    # (không dùng ở bản này)
                 hidden: Sequence[int] = (64, 32, 16),
                 dropout: float = 0.2,
                 use_bn: bool = False):
        super().__init__()
        self.num_users, self.num_items = num_users, num_items

        # Ma trận A học trực tiếp (khởi tạo nhỏ quanh 0 để điểm ~3 sau khi map)
        self.A = nn.Parameter(torch.randn(num_users, num_items) * 0.01)

        # Tower MLP: user nhận vectơ kích thước num_items; item nhận vectơ kích thước num_users
        def make_mlp(in_dim):
            layers = []
            last = in_dim
            for h in hidden:
                layers += [nn.Linear(last, h)]
                if use_bn:
                    layers += [nn.BatchNorm1d(h)]
                layers += [nn.ReLU(inplace=True), nn.Dropout(dropout)]
                last = h
            return nn.Sequential(*layers), last

        self.user_mlp, Du = make_mlp(num_items)
        self.item_mlp, Dv = make_mlp(num_users)
        assert Du == Dv, "Hai tower phải có cùng output dim để tính cosine."
        self.out_dim = Du
        self.head = nn.Linear(self.out_dim * 2, 1)   # học cách ghép pu & qv -> score
        nn.init.xavier_uniform_(self.head.weight)
        nn.init.zeros_(self.head.bias)
        # Khởi tạo tuyến tính
        for m in list(self.user_mlp.modules()) + list(self.item_mlp.modules()):
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    # --------------------------- forward (giống Neu) ---------------------------
    def forward(self, users: torch.Tensor, items: torch.Tensor) -> torch.Tensor:
        """
        users/items: LongTensor (B,)
        return: FloatTensor (B,) in [1, 5]
        """
        # Lấy hàng/cột từ A
        rows = self.A[users, :]                   # (B, num_items)
        cols = self.A[:, items].transpose(0, 1)   # (B, num_users)
    
        # Qua MLP hai nhánh
        pu = self.user_mlp(rows) if len(self.user_mlp) else rows   # (B, D)
        qv = self.item_mlp(cols) if len(self.item_mlp) else cols   # (B, D)
    
        # (tuỳ chọn) normalize nhẹ, có thể bỏ nếu muốn để Linear tự học
        # pu = F.normalize(pu, p=2, dim=-1)
        # qv = F.normalize(qv, p=2, dim=-1)
    
        # Ghép và cho qua Linear head
        z = torch.cat([pu, qv], dim=-1)           # (B, 2D)
        s = self.head(z).squeeze(-1)              # (B,)
    
        # Ràng buộc đầu ra về [1, 5] bằng sigmoid thay vì clamp
        
        return s

    


In [13]:
import torch
import torch.nn as nn
from typing import Optional, Sequence, Tuple

class LightGCN(nn.Module):
    """
    LightGCN tối giản cho implicit CF.
    - E^(0) = concat([E_user, E_item]) với shape [(M+N), d] (user trước, item sau).
    - Propagation: E^(k+1) = A_tilde @ E^(k), A_tilde = D^{-1/2} A D^{-1/2}.
    - Layer-combine: E_final = sum_{k=0..K} alpha_k * E^(k), mặc định alpha_k = 1/(K+1).
    - Score(u,i) = <e_u, e_i>.
    """

    def __init__(
        self,
        num_users: int,
        num_items: int,
        embedding_dim: int = 64,
        num_layers: int = 3,
        edges: Optional[torch.LongTensor] = None,   # shape [2, E], (user_id, item_id)
        alpha: Optional[torch.Tensor] = None,       # (K+1,), nếu None -> uniform
        device: Optional[torch.device] = None,
        dtype: torch.dtype = torch.float32,
    ):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        # E^(0): embedding riêng user & item
        self.user_emb = nn.Embedding(num_users, embedding_dim)
        self.item_emb = nn.Embedding(num_items, embedding_dim)
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)

        # Alpha cho trộn tầng
        no_edges = (edges is None) or (edges.numel() == 0)
        if alpha is None:
            if no_edges:
                alpha = torch.zeros(num_layers + 1, dtype=dtype); alpha[0] = 1.0
            else:
                alpha = torch.full((num_layers + 1,), 1.0/(num_layers + 1), dtype=dtype)
        else:
            assert alpha.numel() == num_layers + 1, "alpha phải có K+1 phần tử"
            alpha = alpha.to(dtype=dtype)
        self.register_buffer("alpha", alpha)

        # A_tilde (sparse)
        if no_edges:
            num_nodes = num_users + num_items
            A_tilde = torch.sparse_coo_tensor(
                torch.zeros((2, 0), dtype=torch.long),
                torch.tensor([], dtype=dtype),
                (num_nodes, num_nodes),
                device=device,
                dtype=dtype
            ).coalesce()
        else:
            A_tilde = self._build_norm_adj(edges, device=device, dtype=dtype)

        # Lưu buffer chỉ số & giá trị để tái tạo nhanh mỗi lần propagate
        self.register_buffer("A_tilde_indices", A_tilde.indices())
        self.register_buffer("A_tilde_values",  A_tilde.values())
        self.A_tilde_size = A_tilde.size()

    # ---------- helpers ----------
    def _build_norm_adj(
        self,
        edges: torch.LongTensor,
        device: Optional[torch.device],
        dtype: torch.dtype,
    ) -> torch.Tensor:
        """Xây A_tilde = D^{-1/2} A D^{-1/2} cho đồ thị hai phía (vô hướng)."""
        M, N = self.num_users, self.num_items
        if device is None:
            device = edges.device

        u = edges[0].to(torch.long)               # [E]
        i = (edges[1].to(torch.long) + M)         # shift item id: [E] trong [M, M+N)

        # cạnh vô hướng: (u,i) & (i,u)
        src = torch.cat([u, i])
        dst = torch.cat([i, u])
        indices = torch.stack([src, dst], dim=0)  # [2, 2E]
        values = torch.ones(indices.size(1), dtype=dtype, device=device)
        num_nodes = M + N

        A = torch.sparse_coo_tensor(indices, values, (num_nodes, num_nodes),
                                    device=device, dtype=dtype).coalesce()

        deg = torch.sparse.sum(A, dim=1).to_dense().clamp(min=1.0)  # tránh 0
        deg_inv_sqrt = deg.pow(-0.5)
        row, col = A.indices()
        norm_vals = deg_inv_sqrt[row] * A.values() * deg_inv_sqrt[col]
        return torch.sparse_coo_tensor(A.indices(), norm_vals, A.size(),
                                       device=device, dtype=dtype).coalesce()

    def _E0(self) -> torch.Tensor:
        """E^(0) = concat([E_user, E_item]) với shape [(M+N), d]."""
        return torch.cat([self.user_emb.weight, self.item_emb.weight], dim=0)

    def _propagate(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """Lan truyền K bước và trộn tầng. Trả về (E_user_final, E_item_final)."""
        E0 = self._E0()
        Es = [E0]

        if self.A_tilde_values.numel() > 0:
            A_tilde = torch.sparse_coo_tensor(
                self.A_tilde_indices, self.A_tilde_values, self.A_tilde_size,
                device=E0.device, dtype=E0.dtype
            )
            Ek = E0
            for _ in range(self.num_layers):
                Ek = torch.sparse.mm(A_tilde, Ek)
                Es.append(Ek)
        else:
            # không cạnh → các tầng sau = 0, alpha[0]=1 ⇒ E_final = E0
            for _ in range(self.num_layers):
                Es.append(torch.zeros_like(E0))

        E_final = torch.zeros_like(E0)
        for k, Ek in enumerate(Es):
            E_final = E_final + self.alpha[k] * Ek

        return E_final[: self.num_users], E_final[self.num_users :]

    # ---------- public API ----------
    @torch.no_grad()
    def get_all_embeddings(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """Trả về (E_user_final [M,d], E_item_final [N,d])."""
        return self._propagate()

    def forward(self, users: torch.Tensor, items: torch.Tensor) -> torch.Tensor:
        """
        users, items: LongTensor [B]
        Trả về: scores [B] = <e_u, e_i>
        """
        U, I = self._propagate()
        eu = U[users]            # [B, d]
        ei = I[items]            # [B, d]
        return (eu * ei).sum(dim=1)

    def bpr_loss(
        self,
        users: torch.Tensor,
        pos_items: torch.Tensor,
        neg_items: torch.Tensor,
        l2_reg: float = 1e-4,
    ) -> torch.Tensor:
        """
        Pairwise BPR loss; regularize chỉ E^(0) như LightGCN gốc.
        users, pos_items, neg_items: LongTensor [B]
        """
        U, I = self._propagate()
        eu = U[users]
        ei = I[pos_items]
        ej = I[neg_items]
        y_pos = (eu * ei).sum(dim=1)
        y_neg = (eu * ej).sum(dim=1)
        loss = -torch.nn.functional.logsigmoid(y_pos - y_neg).mean()

        # L2 chỉ trên E^(0)
        reg = (
            self.user_emb.weight.norm(p=2).pow(2)
            + self.item_emb.weight.norm(p=2).pow(2)
        ) / (self.num_users + self.num_items)

        return loss + l2_reg * reg


In [ ]:
# NeuMF in PyTorch: GMF + MLP for rating regression + AdamW + LR scheduler + save "Id,Score"
import os, math, random
import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split   # <-- thêm import

# ================== Hyperparams & configs ==================
SEED = 42
SUB_PATH   = "output/submission.csv"
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = 'cpu'
K_GMF = 8
K_MLP = 32
MLP_LAYERS = (32, 16, 8)
DROPOUT = 0.4
LR = 1e-2
EPOCHS = 100
BATCH_TRAIN = 1024
BATCH_TEST  = 1024
VAL_RATIO = 0.2
PATIENCE = 20
WEIGHT_DECAY = 1e-4  # AdamW weight decay

# ================== Assume df_train / df_test are ready ==================
# df_train: userid,movieid,rating ; df_test: userid,movieid

# ---- mapping ID -> index liên tục (từ cả train + test) ----
all_users = pd.Index(pd.concat([df_train['userid'], df_test['userid']]).unique())
all_items = pd.Index(pd.concat([df_train['movieid'], df_test['movieid']]).unique())
user2idx = {u:i for i,u in enumerate(all_users)}
item2idx = {m:i for i,m in enumerate(all_items)}
num_users = len(user2idx); num_items = len(item2idx)

df_train_idx = df_train.assign(
    user_idx = df_train['userid'].map(user2idx).astype('int64'),
    item_idx = df_train['movieid'].map(item2idx).astype('int64')
)
df_test_idx = df_test.assign(
    user_idx = df_test['userid'].map(user2idx).astype('int64'),
    item_idx = df_test['movieid'].map(item2idx).astype('int64')
)

min_r = float(df_train['rating'].min())
max_r = float(df_train['rating'].max())
global_mean = float(df_train['rating'].mean())
seen_users = set(df_train['userid'].unique())
seen_items = set(df_train['movieid'].unique())

# ================== Dataset / DataLoader ==================
class RatingsDS(Dataset):
    def __init__(self, u, i, y=None):
        self.u = torch.as_tensor(u, dtype=torch.long)
        self.i = torch.as_tensor(i, dtype=torch.long)
        self.y = None if y is None else torch.as_tensor(y, dtype=torch.float32)
    def __len__(self): return len(self.u)
    def __getitem__(self, idx):
        if self.y is None:
            return self.u[idx], self.i[idx]
        return self.u[idx], self.i[idx], self.y[idx]

train_full = RatingsDS(
    df_train_idx['user_idx'].values,
    df_train_idx['item_idx'].values,
    df_train_idx['rating'].values,
)

all_idx = np.arange(len(df_train_idx))
train_idx, val_idx = train_test_split(
    all_idx,
    test_size=VAL_RATIO,
    random_state=SEED,
    stratify=df_train_idx['rating']
)

from torch.utils.data import Subset
train_ds = Subset(train_full, train_idx)
val_ds   = Subset(train_full, val_idx)

test_ds  = RatingsDS(df_test_idx['user_idx'].values, df_test_idx['item_idx'].values)

train_loader = DataLoader(train_ds, batch_size=BATCH_TRAIN, shuffle=True, drop_last=False)
val_loader   = DataLoader(val_ds, batch_size=max(512, BATCH_TRAIN), shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=BATCH_TEST, shuffle=False)

# ================== Model ==================
df_tr = df_train.copy()

# Nếu có rating và muốn lấy positive theo ngưỡng:
if 'rating' in df_tr.columns:
    df_tr = df_tr[df_tr['rating'] >= 3.0]   # hoặc >= 3.5, tuỳ bài

# Map sang chỉ số liên tục
df_tr['user_idx'] = df_tr['userid'].map(user2idx).astype('int64')
df_tr['item_idx'] = df_tr['movieid'].map(item2idx).astype('int64')

# Mỗi (user,item) một cạnh (LightGCN không cần multiple-edges)
df_tr = df_tr.drop_duplicates(subset=['user_idx', 'item_idx'])

u = torch.as_tensor(df_tr['user_idx'].values, dtype=torch.long)
i = torch.as_tensor(df_tr['item_idx'].values, dtype=torch.long)
edges = torch.stack([u, i], dim=0).to(device)   # shape [2, E]


model = NeuMF(num_users, num_items, K_GMF, K_MLP, MLP_LAYERS, DROPOUT).to(device)
# model = DMF(num_users, num_items, d1=64, hidden=(64,32), dropout=0.3, use_bn=True).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# model = LightGCN(num_users, num_items, embedding_dim=64, num_layers=3, edges=edges.to(device)).to(device)
# ================== Train ==================
device = next(model.parameters()).device
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=3, min_lr=1e-6
)

def rmse(y_true, y_pred):
    return math.sqrt(((y_true - y_pred) ** 2).mean().item())
from sklearn.metrics import r2_score

def r2_torch(y_true: torch.Tensor, y_pred: torch.Tensor) -> float:
    """R^2 dùng sklearn."""
    return r2_score(y_true.detach().cpu().numpy(),
                    y_pred.detach().cpu().numpy())

best_val = float("inf"); patience = PATIENCE; bad = 0
best_state = None

for epoch in range(EPOCHS):
    # ----- train -----
    model.train()
    for u,i,y in tqdm(train_loader):
        u,i,y = u.to(device), i.to(device), y.to(device)
        optimizer.zero_grad()
        p = model(u,i)
        loss = criterion(p, y)
        loss.backward()
        optimizer.step()

    # ----- evaluate train -----
    model.eval()
    ys_tr, ps_tr = [], []
    with torch.no_grad():
        for u,i,y in train_loader:
            u,i,y = u.to(device), i.to(device), y.to(device)
            p = model(u,i)
            ys_tr.append(y.cpu()); ps_tr.append(p.cpu())
    y_tr = torch.cat(ys_tr); p_tr = torch.cat(ps_tr)
    train_rmse = rmse(y_tr, p_tr)
    train_r2   = r2_torch(y_tr, p_tr)

    # ----- validate -----
    ys_v, ps_v = [], []
    with torch.no_grad():
        for u,i,y in tqdm(val_loader):
            u,i,y = u.to(device), i.to(device), y.to(device)
            p = model(u,i)
            ys_v.append(y.cpu()); ps_v.append(p.cpu())
    y_val = torch.cat(ys_v); p_val = torch.cat(ps_v)
    val_rmse = rmse(y_val, p_val)
    val_r2   = r2_torch(y_val, p_val)

    scheduler.step(val_rmse)

    curr_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1:02d} | "
          f"train RMSE: {train_rmse:.4f} | val RMSE: {val_rmse:.4f} | "
          f"train R2: {train_r2:.4f} | val R2: {val_r2:.4f} | LR: {curr_lr:.2e}")

    if val_rmse + 1e-6 < best_val:
        best_val = val_rmse; bad = 0
        best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping.")
            break

# Load best
if best_state is not None:
    model.load_state_dict(best_state)
model.to(device); model.eval()

# ================== Predict test ==================
preds = []
with torch.no_grad():
    for u,i in test_loader:
        u,i = u.to(device), i.to(device)
        p = model(u,i).cpu().numpy()
        preds.append(p)
pred = np.concatenate(preds, axis=0).reshape(-1)

# clip về [min_r, max_r]
pred = np.clip(pred, min_r, max_r)

# fallback cho cold-start
mask_cold = (~df_test['userid'].isin(seen_users)) | (~df_test['movieid'].isin(seen_items))
if mask_cold.any():
    pred[mask_cold.values] = global_mean

# ================== Save "Id,Score" (Id bắt đầu từ 1) ==================
submission = pd.DataFrame({
    "Id": np.arange(1, len(df_test) + 1, dtype=int),
    "Score": pred.astype(float)
})
submission.to_csv(SUB_PATH, index=False)
print(f"Saved to: {os.path.abspath(SUB_PATH)}")
print(submission.head())


100%|██████████| 18/18 [00:01<00:00, 17.25it/s]


Epoch 01 | train RMSE: 1.3811 | val RMSE: 1.3949 | train R2: -0.5043 | val R2: -0.5342 | LR: 1.00e-02


100%|██████████| 18/18 [00:01<00:00, 16.32it/s]


Epoch 02 | train RMSE: 1.1897 | val RMSE: 1.2226 | train R2: -0.1162 | val R2: -0.1786 | LR: 1.00e-02


100%|██████████| 18/18 [00:01<00:00, 16.85it/s]


...

Epoch 49 | train RMSE: 0.4877 | val RMSE: 0.9059 | train R2: 0.8124 | val R2: 0.3529 | LR: 3.13e-04


100%|██████████| 18/18 [00:01<00:00, 15.26it/s]


Epoch 50 | train RMSE: 0.4870 | val RMSE: 0.9061 | train R2: 0.8130 | val R2: 0.3527 | LR: 3.13e-04


100%|██████████| 18/18 [00:01<00:00, 16.80it/s]


Epoch 51 | train RMSE: 0.4862 | val RMSE: 0.9061 | train R2: 0.8135 | val R2: 0.3526 | LR: 1.56e-04
Early stopping.
Saved to: e:\Sao lưu onedrive\not de ra truong nao\Hệ gợi ý\Project_RS_in_HUST\output\submission.csv
   Id     Score
0   1  3.898017
1   2  3.646442
2   3  4.296011
3   4  4.007053
4   5  2.027400
